In [1]:
import torch 
import torch.nn as nn
import numpy as np
from transformers import GPT2Model, GPT2Config
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

/user/ae2848/.conda/envs/aymenWorld/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
objective = 'scale'
method = 'vanilla'
if objective == 'scale':
    n_dims = 10
    num_tasks = 512
    num_tasks_val = 32
    n_points = 500
    n_positions = 500
    batch_size = 32 # or 16 to reduce gpu memory usage, especially for cid method
    epochs = 10000
    print_inc = 100
    n_points_curriculum = {
        'start':100,
        'end':n_points,
        'inc':50,
        'schedule':1000
    }
    n_dims_curriculum = {   
        'start':2,
        'end':n_dims,
        'inc':1,
        'schedule':1000
    }
else:
    n_dims = 10
    num_tasks = 64
    num_tasks_val = 16
    n_points = 20
    n_positions = 20
    assert n_positions >= n_points
    epochs = 100
    batch_size = 4
    print_inc = 10
    n_points_curriculum = {
        'start':10,
        'end':n_points,
        'inc':10,
        'schedule':10
    }
    n_dims_curriculum = {   
        'start':2,
        'end':n_dims,
        'inc':1,
        'schedule':10
    }
assert n_positions >= n_points
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
class GaussianSampler():
    def __init__(self, n_dims, bias=None, scale=None):
        self.bias = bias
        self.scale = scale
        self.n_dims = n_dims

    def sample_xs(self, n_points, num_tasks, n_dims_truncated=None, seeds=None):
        if seeds is None:
            xs_b = torch.randn(num_tasks, n_points, self.n_dims)
        else:
            xs_b = torch.zeros(num_tasks, n_points, self.n_dims)
            generator = torch.Generator()
            assert len(seeds) == num_tasks
            for i, seed in enumerate(seeds):
                generator.manual_seed(seed)
                xs_b[i] = torch.randn(n_points, self.n_dims, generator=generator)
        if self.scale is not None:
            xs_b = xs_b @ self.scale
        if self.bias is not None:
            xs_b += self.bias
        if n_dims_truncated is not None:
            xs_b[:, :, n_dims_truncated:] = 0
        return xs_b

In [4]:
class LinearRegression():
    def __init__(self, n_dims, num_tasks, pool_dict=None, seeds=None, scale=1):
        self.scale = scale

        if pool_dict is None and seeds is None:
            self.w_b = torch.randn(num_tasks, n_dims, 1)
        elif seeds is not None:
            self.w_b = torch.zeros(num_tasks, n_dims, 1)
            generator = torch.Generator()
            assert len(seeds) == num_tasks
            for i, seed in enumerate(seeds):
                generator.manual_seed(seed)
                self.w_b[i] = torch.randn(n_dims, 1, generator=generator)
        else:
            assert "w" in pool_dict
            indices = torch.randperm(len(pool_dict["w"]))[:num_tasks]
            self.w_b = pool_dict["w"][indices]

    def evaluate(self, xs_b):
        w_b = self.w_b.to(xs_b.device)
        ys_b = self.scale * (xs_b @ w_b)[:, :, 0]
        return ys_b

    @staticmethod
    def generate_pool_dict(n_dims, num_tasks):  
        return {"w": torch.randn(num_tasks, n_dims, 1)}

    @staticmethod
    def get_metric():
        return lambda ys_pred, ys : (ys - ys_pred).square()

    @staticmethod
    def get_training_metric():
        return lambda ys_pred, ys : (ys - ys_pred).square().mean()

In [5]:
class TransformerModel(nn.Module):
    def __init__(self, n_dims, n_positions, n_embd=128, n_layer=12, n_head=4):
        super(TransformerModel, self).__init__()
        configuration = GPT2Config(
            n_positions=2 * n_positions,
            n_embd=n_embd,
            n_layer=n_layer,
            n_head=n_head,
            resid_pdrop=0.0,
            embd_pdrop=0.0,
            attn_pdrop=0.0,
            use_cache=False,
        )

        self.n_positions = n_positions
        self.n_dims = n_dims
        self._read_in = nn.Linear(n_dims, n_embd)
        self._backbone = GPT2Model(configuration)
        self._read_out = nn.Linear(n_embd, 1)

    @staticmethod
    def _combine(xs_b, ys_b):
        """Interleaves the x's and the y's into a single sequence."""
        batch , points, dim = xs_b.shape
        ys_b_wide = torch.cat(
            (
                ys_b.view(batch, points, 1),
                torch.zeros(batch, points, dim - 1, device=ys_b.device),
            ),
            axis=2,
        )
        zs = torch.stack((xs_b, ys_b_wide), dim=2)
        zs = zs.view(batch, 2 * points, dim)
        return zs

    def forward(self, xs, ys):
        zs = self._combine(xs, ys)
        embeds = self._read_in(zs)
        output = self._backbone(inputs_embeds=embeds).last_hidden_state
        prediction = self._read_out(output)
        return prediction[:, ::2, 0]  # predict only on xs

In [6]:
class Curriculum:
    def __init__(self, n_points_curriculum,n_dims_curriculum):
        self.n_points_curriculum = n_points_curriculum
        self.n_dims_curriculum = n_dims_curriculum
        self.n_points = n_points_curriculum['start']
        self.n_dims_truncated = n_dims_curriculum['start']
        self.step_count = 0

    def update(self):
        self.step_count += 1
        self.n_dims_truncated = self.update_var(
            self.n_dims_truncated, self.n_dims_curriculum
            )
        self.n_points = self.update_var(
            self.n_points, self.n_points_curriculum
            )

    def update_var(self, var, schedule):
        if self.step_count % schedule['schedule'] == 0:
            var += schedule['inc']
        return min(var, schedule['end'])

In [7]:
class TrainDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        x_sample = self.X[index]
        y_sample = self.Y[index]
        return x_sample, y_sample

In [8]:
class CopulaDensityModel(nn.Module): # copula density composed with exp function not copula 
    def __init__(self):
        super(CopulaDensityModel, self).__init__()
        self.backbone = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(), # output in [0,1]
        ) # think about forcing other structure
    def forward(self,u,v): # u,v are 3D tensors : batch x (points-1) x dim
        return self.backbone(torch.cat([u.reshape(-1,1),v.reshape(-1,1)],dim=-1)).reshape(u.shape)

In [9]:
class CidTransformerModel(nn.Module):
    def __init__(self, n_dims, n_positions, n_embd=128, n_layer=12, n_head=4):
        super(CidTransformerModel, self).__init__()
        configuration = GPT2Config(
            n_positions= 2*n_positions, # maybe n_positions (reduce gpu memory)
            n_embd=n_embd,
            n_layer=n_layer,
            n_head=n_head,
            resid_pdrop=0.0,
            embd_pdrop=0.0,
            attn_pdrop=0.0,
            use_cache=False,
        )

        self.n_positions = n_positions
        self.n_dims = n_dims
        self._read_in = nn.Linear(1, n_embd)
        self._backbone = GPT2Model(configuration)
        self._read_out = nn.Linear(n_embd, 2)

    @staticmethod
    def _combine(xs_b, ys_b):
        """Interleaves the x's and the y's into a single sequence of 1D tensors."""
        batch, points, dim = xs_b.shape
        zs = torch.cat((xs_b, ys_b[:,:,None]), dim=2)
        zs = zs.view(batch, points*(dim+1),1)
        return zs

    def forward(self, xs, ys):
        _, points, dim = xs.shape
        zs = self._combine(xs, ys)
        embeds = self._read_in(zs)
        output = self._backbone(inputs_embeds=embeds).last_hidden_state
        mean  = self._read_out(output)[:,:,0]
        std = torch.exp(self._read_out(output)[:,:,1])
        log_prediction = -0.5*((zs[:,:,0]-mean)/std)**2 - torch.log(std*np.sqrt(2*np.pi))
        filtered_prediction = torch.arange(points*(dim+1))%(dim+1)==(dim-1)
        return mean[:,filtered_prediction], std[:,filtered_prediction], log_prediction[:,torch.arange(points*(dim+1))%(dim+1)!=dim]

In [10]:
class CidModel(nn.Module):
    def __init__(self, n_dims, n_positions):
        super(CidModel, self).__init__()
        self.copulaYmodel = CopulaDensityModel().to(device)
        self.copulaXmodel = CopulaDensityModel().to(device)
        self.cidTransformerModel = CidTransformerModel(n_dims=n_dims,n_positions=(n_dims+1)*n_positions).to(device)
        self.alphas = 1/(torch.arange(end=n_positions-1,device=device)+1)
    def forward(self,xs,ys): 
        '''
        batch , points, dim = xs.shape
        log_prediction = self.cidTransformerModel(xs,ys)[-1] # num_tasks x (n_points*n_dims)
        log_prediction = log_prediction.view(batch,points,dim) # to check if this is correct
        log_ppd = torch.zeros(batch).to(device)
        log_ppds = []
        for i in range(points-1): # too slow 
            alpha = 1/(i+1)
            log_copula_prod_x = 0
            for j in range(dim):
                u_ij = log_prediction[:,i+1,j:j+1]
                v_ij = log_prediction[:,i,j:j+1]
                log_copula_prod_x += torch.log(self.copulaXmodel(u_ij,v_ij)[:,0])
            copula_prod_x = torch.exp(log_copula_prod_x)
            q_i = log_prediction[:,i+1,-1:]
            r_i = log_prediction[:,i,-1:]
            copula_prod_y = self.copulaYmodel(q_i,r_i)[:,0]
            log_ppd += torch.log(1-alpha+alpha*copula_prod_x*copula_prod_y) - torch.log(1-alpha+alpha*copula_prod_x)
            log_ppds.append(log_ppd)
        log_ppds = torch.stack(log_ppds,dim=1)
        return log_ppds,log_prediction[:,:,-1]    
        '''
        batch , points, dim = xs.shape
        log_prediction = self.cidTransformerModel(xs,ys)[-1] # batch x (points*dims)
        log_prediction = log_prediction.view(batch,points,dim) # batch x points x dim 
        log_prediction_f = log_prediction[:,1:,:] #.transpose(1,2)  # batch x (points-1) x dim 
        log_prediction = log_prediction[:,:-1,:] #.transpose(1,2) # batch x (points-1) x dim
        copulaXfactor = torch.exp(torch.sum(torch.log(self.copulaXmodel(log_prediction_f,log_prediction)),dim=-1)) # batch x (points-1)
        copulaYfactor = self.copulaYmodel(log_prediction_f[:,:,-1],log_prediction[:,:,-1]) # batch x (points-1)
        alphas = self.alphas[:points-1]
        result = torch.log(1-alphas+alphas*copulaYfactor*copulaXfactor)-torch.log(1-alphas+alphas*copulaXfactor) # batch x (points-1)
        return torch.cumsum(result,dim=-1),log_prediction[:,:,-1]    

In [12]:
class CidLoss(nn.Module):
    def __init__(self):
        super(CidLoss, self).__init__()
    def forward(self, log_ppds,log_predictions):
        loss_value = (log_ppds-log_predictions).square().mean()
        return loss_value

In [13]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [14]:
class DynamicPlotter:
    def __init__(self):
        self.train_losses = []
        self.val_losses = []
        self.steps = []

    def update_plot(self, step, train_loss, val_loss):
        
        self.steps.append(step)
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)

        plt.clf()
        plt.subplot(2,1,1)
        plt.plot(self.steps, self.train_losses, label='Train Loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()

        plt.subplot(2,1,2)
        plt.plot(self.steps, self.val_losses, label='Val Loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()

        #self.fig.canvas.draw()
        #plt.show(block=False)
        plt.tight_layout()
        plt.pause(0.1)
        plt.show()


In [15]:
curriculum = Curriculum(n_points_curriculum,n_dims_curriculum)

In [16]:
seeds_ws = list(range(1,1+num_tasks_val))
seeds_xs = list(range(1+num_tasks_val,1+2*num_tasks_val))
data_sampler = GaussianSampler(n_dims)
#pool_dict = LinearRegression.generate_pool_dict(n_dims, num_tasks)
task = LinearRegression(n_dims, num_tasks_val, seeds=seeds_ws)
val_xs = data_sampler.sample_xs(n_points,num_tasks_val,n_dims,seeds=seeds_xs).to(device)
val_ys = task.evaluate(val_xs).to(device)

In [17]:
seeds_ws = list(range(2*num_tasks_val+1,num_tasks+2*num_tasks_val+1))
seeds_xs = list(range(2*num_tasks_val+1+num_tasks,2*num_tasks_val+1+2*num_tasks))
data_sampler = GaussianSampler(n_dims)
#pool_dict = LinearRegression.generate_pool_dict(n_dims, num_tasks)
task = LinearRegression(n_dims, num_tasks, seeds=seeds_ws) 
X = data_sampler.sample_xs(curriculum.n_points,num_tasks,curriculum.n_dims_truncated,seeds=seeds_xs)
Y = task.evaluate(X)
dataset = TrainDataset(X,Y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
#dynamic_plotter = DynamicPlotter()
if method == 'vanilla':
    vanillaTransformerModel = TransformerModel(n_dims=n_dims,n_positions=n_positions).to(device)
    print(f"Model size: {get_model_size(vanillaTransformerModel)} parameters")
    optimizer = torch.optim.Adam(vanillaTransformerModel.parameters(), lr=1e-3)
    loss_func = nn.CrossEntropyLoss() #nn.MSELoss()
    writer = SummaryWriter(comment=f'{objective}_{method}_transformer')
    for epoch in range(epochs):
        train_loss = 0
        for xs, ys in dataloader:
            vanillaTransformerModel.train()
            optimizer.zero_grad()
            output = vanillaTransformerModel(xs.to(device), ys[:,:,None].to(device))
            loss = loss_func(output, ys.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().item()

        vanillaTransformerModel.eval()
        with torch.no_grad():
            output = vanillaTransformerModel(val_xs, val_ys[:,:,None])
            val_loss = loss_func(output, val_ys).detach().item()

        if epoch % print_inc == 0:
            print(f'Epoch {epoch}: Train Loss: {train_loss}, Val Loss: {val_loss}')
            #dynamic_plotter.update_plot(epoch , train_loss, val_loss)

        writer.add_scalar('Train/Loss', train_loss, global_step=epoch)
        writer.add_scalar('Val/Loss', val_loss, global_step=epoch)
        curriculum.update()


elif method == 'cid':
    cidModel = CidModel(n_dims=n_dims,n_positions=n_positions).to(device) 
    print(f"Model size: {get_model_size(cidModel)} parameters")
    optimizer = torch.optim.Adam(cidModel.parameters(), lr=1e-3)
    loss_func = CidLoss() 
    writer = SummaryWriter(comment=f'{objective}_{method}_transformer')
    for epoch in range(epochs):
        train_loss = 0
        for xs, ys in dataloader:
            cidModel.train()
            optimizer.zero_grad()
            log_ppds,log_predictions = cidModel(xs.to(device), ys.to(device))
            loss = loss_func(log_ppds,log_predictions[:,:-1])
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().item()

        cidModel.eval()
        with torch.no_grad():
            mean,std,_ = cidModel.cidTransformerModel(val_xs, val_ys)
            output = mean + std*torch.randn(num_tasks_val,n_points).to(device)
            val_loss = nn.CrossEntropyLoss()(output, val_ys).detach().item()

        if epoch % print_inc == 0:
            print(f'Epoch {epoch}: Train Loss: {train_loss}, Val Loss: {val_loss}')
            #dynamic_plotter.update_plot(epoch , train_loss, val_loss)
        writer.add_scalar('Train/Loss', train_loss, global_step=epoch)
        writer.add_scalar('Val/Loss', val_loss, global_step=epoch)
        curriculum.update()

Model size: 8941953 parameters
Epoch 0: Train Loss: -67.4660997390747, Val Loss: -45.69856262207031
Epoch 100: Train Loss: -1388.0510139465332, Val Loss: -58.177886962890625
Epoch 200: Train Loss: -6558.633407592773, Val Loss: -1634.582275390625
Epoch 300: Train Loss: -2477.7537155151367, Val Loss: -6243.47607421875
Epoch 400: Train Loss: -87.23016834259033, Val Loss: -68.99769592285156
Epoch 500: Train Loss: -615.826488494873, Val Loss: -517.13427734375
Epoch 600: Train Loss: -41271.10986328125, Val Loss: 741.832763671875
Epoch 700: Train Loss: -81674.32482910156, Val Loss: 28168.970703125
Epoch 800: Train Loss: -118046.716796875, Val Loss: -65232.44140625
Epoch 900: Train Loss: -401545.7109375, Val Loss: -50170.8671875
Epoch 1000: Train Loss: -549630.4892578125, Val Loss: -56925.8359375
Epoch 1100: Train Loss: -936157.755859375, Val Loss: 11174.271484375
Epoch 1200: Train Loss: -1248708.625, Val Loss: -211022.4375
Epoch 1300: Train Loss: -1711628.740234375, Val Loss: -267674.5625
Epo

In [41]:
#!tensorboard --logdir='./' --host localhost --port 8088